Knjižnice

In [1]:
import csv
from csv import DictReader
import re
import matplotlib
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import numpy as np
import sys

Branje podatkov

In [2]:
reader = DictReader(open('podatki/min_place.csv', 'rt', encoding='utf-8'))

leto_min_placa = dict()

for row in reader:
    leto_min_placa[row["Leto"]] = row["Placa"];

print(leto_min_placa)

{'2010': '734.15', '2011': '748.10', '2012': '763.06', '2013': '783.66', '2014': '789.15', '2015': '790.73', '2016': '790.73', '2017': '804.96', '2018': '842.79', '2019': '886.63', '2020': '940.58', '2021': '1024.24'}


In [3]:
reader = DictReader(open('podatki/st_in_vrednost_nepremicnin.csv', 'rt', encoding='utf-8'))

tabela_nepr = dict()

for col in reader.fieldnames:
    tabela_nepr[col] = []
    for row in reader:
        tabela_nepr[col].append(row[col])
    reader = DictReader(open('podatki/st_in_vrednost_nepremicnin.csv', 'rt', encoding='utf-8'))


In [4]:
reader = DictReader(open('podatki/povp_place.csv', 'rt'))

meseci, bruto, bruto3, neto, neto3 = [],[],[],[],[]



for row in reader:
    meseci.append(row["MESEC"])
    bruto.append(row["Bruto plaèa Plaèa za mesec [EUR]"])
    bruto3.append(row["Bruto plaèa Plaèa za tromeseèje [EUR]"])
    neto.append(row["Neto plaèa Plaèa za mesec [EUR]"])
    neto3.append(row["Neto plaèa Plaèa za tromeseèje [EUR]"])

povp_place = np.array([meseci,bruto,bruto3,neto,neto3])
del meseci,bruto,bruto3,neto,neto3


In [5]:
#!!!!!!PAZI TABELA IMA VREDNOSTI V STRING FORMATU, NAREDI PRETVORBO ČE JO RABIŠ!!!!!!!!!!


#primer uporabe... stolpci tabele : meseci, bruto, povp zadnjih 3 bruto, neto, povp zadnjih 3 neto
for placa in povp_place[1,:3]: #bruto place za prve 3 vnose
    print(placa)

#koliko je placa specificen datum
msk = povp_place[0] == "2014M08"
povp_place[:,msk] #izpisi vse

1539.93
1525.86
1531.61


array([['2014M08'],
       ['1522.25'],
       ['1530.00'],
       ['996.39'],
       ['1000.58']], dtype='<U7')

In [6]:
def EtnRead(year, tip, header = None): #header je če nas zanima le header vrstica, določene ETN datoteke
    if tip not in ('posli','delistavb','zemljisca'): #table get check
        print(f'Unknown file name "{tip}";\nAllowed : "posli" , "delistavb", "zemljisca"')
        return None

    if not (2006 < year < 2023):  #year range check
        print(f'Incorrect year: {year}\nRange is [2007,2022]')
        return None
                            #index stolpca opombe, ki ga zbrišemo saj nam opomba lahko bloata np table
                            #index opombe v csvju : posli = zemljisca = 9 , delistavb = 27
                            ####če je treba preskočiti še kakšen stolpec, dodaj v pravi list index stolpca###
    indexi = [9]            
    if tip == 'delistavb':
        indexi = [27]
        
    if tip == 'zemljisca':
        pass        #preskoci


    data_path = f'podatki/ETN_SLO_CSV_A_KUP/ETN_SLO_KUP_{year}_20220326/ETN_SLO_KUP_{year}_{tip}_20220326.csv'

    with open(data_path,encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        data = []

        head = next(reader,None) #skip header

        if header:
            return head #vrne header row

        for row in reader:
            try:
                for i in indexi:
                    row[i] = ""
            except Exception as e:
                print(row)
                print(e)
                return
            data.append(row)
    
    return np.array(data)


In [7]:

posli2010 = EtnRead(2010,'posli')
zemljisca2010 = EtnRead(2010,'zemljisca')
delistavb2010 = EtnRead(2010,'delistavb')


In [30]:
#EtnRead(2010,'posli',True)
EtnRead(2010,'zemljisca',True)
#EtnRead(2010,'delistavb',True)

['ID Posla',
 'Šifra KO',
 'Ime KO',
 'Občina',
 'Parcelna številka',
 'Vrsta zemljišča',
 'Vrsta trajnega nasada',
 'Starost trajnega nasada',
 'Prodani delež parcele',
 'Opombe o nepremičnini',
 'Površina parcele',
 'Pogodbena cena parcele',
 'Stopnja DDV parcele']

In [41]:
print(zemljisca2010[1,8])



369/1000
0.369


In [65]:
dict_posli = {}
for row in posli2010:
    if row[4] == '':  # cene 0 smiselne?
        continue
    dict_posli[row[0]] = float(row[4].replace(",","."))

In [73]:
dict_zemljisca = {}
for row in zemljisca2010:
    if row[10] == '' or int(row[10]) == '0': # če ni površine preskoči....dodaj morda za površina = 0
        continue
    if row[5] not in (1,2,3,4): #ce ni primernega tipa za stanovanje oz hišo nas ne zanima
        pass
        #continue
    if row[8] != 'NP' and row[8] != '6179844811' and row[8] != '': #preveri pomen NP, ali je Ni Podatka ali kaj
        ops = row[8].split("/") 
    else:
        ops = ['1', '1']
    
    if ops[0] == '':
        ops[0] = 1
    if ops[1] == '':
        ops[1] = 1

    prodanDelez = int(ops[0]) / int(ops[1])

    if prodanDelez == 0:
        continue


    povrsinaParcele = float(row[10].replace(",","."))
    if row[0] in dict_zemljisca:
        dict_zemljisca[row[0]]+= povrsinaParcele * prodanDelez
    else:
        dict_zemljisca[row[0]] = povrsinaParcele * prodanDelez

In [77]:
cenaNaMeter = []
brezPov = 0
for pogodbaId, cena in dict_posli.items():
    if pogodbaId in dict_zemljisca:
        cenaNaMeter.append(cena / dict_zemljisca[pogodbaId])
    else:
        brezPov += 1


In [79]:
stPojav = len(cenaNaMeter)
vsota = sum(cenaNaMeter)
vsota/stPojav

454.6104755610275

In [8]:
def sizeof_fmt(num, suffix="B"): #funkcija za formatiranje izpisa velikosti spremenljivk
    for unit in ["", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"]:
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"

#preveri katerih podatkov ne more prebrati 
def checkRAM(ime):
    for i in range(2007,2023):
        try:
            data = EtnRead(i,ime)
            size = sizeof_fmt(sys.getsizeof(data))
            print(f"{i} => {data.dtype}, ram usage: {size}") #preveri tip tabele ie string length
            del data
        except Exception as e:
            print(f"{i} => {e}")  #prikazi katero leto ti vrže (memoryError -> zmankal rama) exception 
            pass

def test():
    print('delistavb')
    checkRAM('delistavb')
    print()

    print('zemljisca')
    checkRAM('zemljisca')
    print()

    print('posli')
    checkRAM('posli')
    


In [9]:
test()  #izbrisi/zakomentiraj po testiranju


delistavb
2007 => <U80, ram usage: 143.4MiB
2008 => <U86, ram usage: 380.4MiB
2009 => <U289, ram usage: 929.8MiB
2010 => <U289, ram usage: 1.7GiB
2011 => <U289, ram usage: 1.7GiB
2012 => <U289, ram usage: 1.5GiB
2013 => <U289, ram usage: 1.1GiB
2014 => <U222, ram usage: 1.1GiB
2015 => <U181, ram usage: 673.6MiB
2016 => <U196, ram usage: 846.8MiB
2017 => <U180, ram usage: 856.3MiB
2018 => <U179, ram usage: 774.0MiB
2019 => <U163, ram usage: 751.1MiB
2020 => <U161, ram usage: 678.6MiB
2021 => <U446, ram usage: 2.0GiB
2022 => <U161, ram usage: 193.6MiB

zemljisca
2007 => <U28, ram usage: 14.6MiB
2008 => <U32, ram usage: 114.3MiB
2009 => <U29, ram usage: 51.4MiB
2010 => <U29, ram usage: 65.5MiB
2011 => <U29, ram usage: 69.1MiB
2012 => <U29, ram usage: 75.6MiB
2013 => <U29, ram usage: 67.2MiB
2014 => <U29, ram usage: 76.2MiB
2015 => <U29, ram usage: 71.2MiB
2016 => <U29, ram usage: 79.8MiB
2017 => <U29, ram usage: 88.1MiB
2018 => <U29, ram usage: 86.4MiB
2019 => <U29, ram usage: 96.5MiB
202

Analiza podatkov

In [10]:
stolpci = tabela_nepr.keys() #vsi stolpci


vrstica = tabela_nepr["DWELLINGS"] #prvi stolpec, pove kaj pomeni posamezna vrstica

print(vrstica) #izpiši prvi stolpec

zanimiva_vr = vrstica.index("1 Dwellings - TOTAL")  #pridobi vrstico, ki te zanima

print(zanimiva_vr)


['1 Dwellings - TOTAL', '1.1 Newly built dwellings', '1.1.1 Newly built flats', '1.1.2 Newly built family houses', '1.2 Existing dwellings', '1.2.1 Existing flats, Slovenia', '1.2.2 Existing family houses']
0


In [11]:


for key in tabela_nepr.keys():
    if re.search("Number of transactions$",key): #primer za iteracijo po # transakcij
        print(key)
        print(tabela_nepr[key][zanimiva_vr]) 

2010Q1 Number of transactions
1961
2010Q2 Number of transactions
2068
2010Q3 Number of transactions
1813
2010Q4 Number of transactions
1939
2011Q1 Number of transactions
1874
2011Q2 Number of transactions
1852
2011Q3 Number of transactions
1851
2011Q4 Number of transactions
2057
2012Q1 Number of transactions
2729
2012Q2 Number of transactions
2718
2012Q3 Number of transactions
2481
2012Q4 Number of transactions
2643
2013Q1 Number of transactions
2345
2013Q2 Number of transactions
2601
2013Q3 Number of transactions
2008
2013Q4 Number of transactions
2285
2014Q1 Number of transactions
2345
2014Q2 Number of transactions
2703
2014Q3 Number of transactions
2525
2014Q4 Number of transactions
2791
2015Q1 Number of transactions
2627
2015Q2 Number of transactions
3078
2015Q3 Number of transactions
2949
2015Q4 Number of transactions
3376
2016Q1 Number of transactions
3286
2016Q2 Number of transactions
3787
2016Q3 Number of transactions
3239
2016Q4 Number of transactions
3706
2017Q1 Number of tra